# FC25 Rush Detection
## Model Preparation 2025-02-06
This notebook will assist in preparing data for training a model which will detect objects in a screenshot of the game. This notebook assumes an initial model has been trained from ```rush-detection-model.ipynb```.

### Flow
1. Gather new screenshots using this project
2. Run inference from existing model against new screenshots
3. Automatically import images and labels into the Label Studio project through API
4. Manually verify and append labels
5. Repeat

### Goal
Using this method, I intend to gather:
[ ] 1000 examples of ball
[ ] 1000 examples of players
[ ] 1000 examples of user controlled player

## Next Steps
After achieving reasonable performance with this model, detection can be extended to pick up a few new objects.

In [ ]:
%%capture
# Environment Setup
%pip install huggingface_hub numpy python-dotenv ultralytics

## Gather New Screenshots
Make sure to run ```screenshot-to-jpg.ipynb``` and ```batch_crop_and_resize_to_height()``` in ```crop-screenshots.ipynb``` to normalize the screenshots to 1080p and crop them to the center of the captured window.  
<br>
Output directory should be ```'../screenshots/jpg/cropped'```

In [ ]:
# 1. Gather new screenshot
new_screenshots_location = "../screenshots/jpg/cropped"

## Inference
Run inference on new screenshots using existing model

In [ ]:
import os

###################################################################
output_location = os.path.expanduser("~/Documents/Rush/inference")
os.makedirs(output_location, exist_ok=True)
###################################################################

In [ ]:
import os
import json
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

###########################################################
model_path = os.path.expanduser("~/Documents/Rush/best.pt")
###########################################################

# Load the trained YOLO model
model = YOLO(model_path)  # Replace with the correct path to your best weights file.

# List all image files in the directory
image_files = [f for f in os.listdir(new_screenshots_location) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Function to perform inference, display results, and save annotations
def run_inference(image_path, save_dir):
    # Load image using OpenCV
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    
    # Perform inference
    results = model(img_rgb, conf=0.2)
    
    # Render results on the image
    annotated_img = results[0].plot()  # Annotate the image with results

    # Save annotated image
    annotated_image_path = os.path.join(save_dir, os.path.basename(image_path))
    plt.imsave(annotated_image_path, annotated_img)  # Save using imsave

    # Prepare detection results for Label Studio
    detections = []
    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])  # Extract coordinates
        confidence = result.conf[0].item()
        class_id = int(result.cls[0].item())

        # Append detection to list
        detections.append({
            "rectanglelabels": results[0].names[class_id],  # Replace with your class names
            "points": {
                "x": x1 / img.shape[1],  # Convert to relative coordinates
                "y": y1 / img.shape[0],  # Convert to relative coordinates
                "width": (x2 - x1) / img.shape[1],
                "height": (y2 - y1) / img.shape[0]
            },
            "confidence": confidence,
            "class_id": class_id  # Class ID of detected object
        })

    # Save detection results in a JSON file
    json_output_path = os.path.join(save_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}.json")
    with open(json_output_path, 'w') as json_file:
        json.dump(detections, json_file, indent=4)

# Run inference for each image in the directory
for image_file in image_files:
    image_path = os.path.join(new_screenshots_location, image_file)
    run_inference(image_path, output_location)

## Label Studio Import
Import the inference results into Label Studio using the API. This assumes you have setup .env in the root of this project with the following keys:
```
LABEL_STUDIO_URL=http://localhost:8080
LABEL_STUDIO_API_TOKEN={api_token}
LABEL_STUDIO_PROJECT_ID={project_id}
```

In [ ]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the .env file located two directories up
env_path = os.path.join(current_dir, '..', '.env')

# Load the .env file
load_dotenv(env_path)

# Access values
label_studio_url = os.getenv('LABEL_STUDIO_URL')
label_studio_api_token = os.getenv('LABEL_STUDIO_API_TOKEN')
label_studio_project_id = os.getenv('LABEL_STUDIO_PROJECT_ID')

# Create a dictionary of the variables to check
env_values = {
    "Label Studio URL": label_studio_url,
    "Label Studio API Token": label_studio_api_token,
    "Label Studio Project ID": label_studio_project_id,
}

# Check each variable and print its status
for name, value in env_values.items():
    if value:
        print(f"{name} is set.")
    else:
        print(f"{name} is not set.")

In [ ]:
import os
import json
import requests
from tqdm import tqdm

# API Headers
HEADERS = {
    "Authorization": f"Token {label_studio_api_token}"
}

def upload_image(image_path):
    """Uploads an image to Label Studio and returns its task_id."""
    with open(image_path, "rb") as f:
        response = requests.post(
            f"{label_studio_url}/api/projects/{label_studio_project_id}/import?return_task_ids=true",
            headers=HEADERS,
            files={"file": f}
        )

    if response.status_code == 201:
        task_id = response.json().get("task_ids", [None])[0]
        if task_id:
            return task_id  # Return the task ID
    print(f"❌ Image upload failed for {image_path}: {response.text}")
    return None

def upload_tasks(image_dir, label_dir):
    """Uploads images and their corresponding JSON annotations to Label Studio correctly."""
    
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

    for image_file in tqdm(image_files, desc="Uploading images"):
        image_path = os.path.join(image_dir, image_file)
        annotation_path = os.path.join(label_dir, os.path.splitext(image_file)[0] + ".json")

        if not os.path.exists(annotation_path):
            print(f"⚠️ No annotation found for {image_file}. Skipping...")
            continue

        # Upload image and get its task ID
        task_id = upload_image(image_path)
        if not task_id:
            continue  # Skip if upload failed

        # Load annotations
        with open(annotation_path, "r") as f:
            annotations = json.load(f)

        # ✅ Fix: Convert coordinates to percentages
        formatted_annotations = []
        for ann in annotations:
            formatted_annotations.append({
                "original_width": 1920,  # Set based on actual image dimensions
                "original_height": 1080,
                "image_rotation": 0,
                "from_name": "label", 
                "to_name": "image",
                "type": "rectanglelabels",
                "value": {
                    "x": ann["points"]["x"] * 100,  # Convert to percentage
                    "y": ann["points"]["y"] * 100,
                    "width": ann["points"]["width"] * 100,
                    "height": ann["points"]["height"] * 100,
                    "rotation": 0,
                    "rectanglelabels": [ann["rectanglelabels"]] 
                }
            })

        annotation_payload = {
            "result": formatted_annotations,
            "was_cancelled": False,
            "ground_truth": False
        }

        # Create a new annotation for the task
        response = requests.post(
            f"{label_studio_url}/api/tasks/{task_id}/annotations/",
            headers={**HEADERS, "Content-Type": "application/json"},
            json=annotation_payload
        )

        if response.status_code >= 300:
            print(f"❌ Error creating annotation for task {task_id}: {response.text}")

# Run the function
upload_tasks(new_screenshots_location, output_location)
